In [ ]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import re
import torch
import esm
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Loading ESM model...")
model_esm, alphabet = esm.pretrained.esm1_t34_670M_UR50S()
batch_converter = alphabet.get_batch_converter()
model_esm.eval()
model_esm.to(device)
print("ESM model loaded.")

Using device: cuda
Loading ESM model...


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1_t34_670M_UR50S.pt" to /root/.cache/torch/hub/checkpoints/esm1_t34_670M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1_t34_670M_UR50S-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm1_t34_670M_UR50S-contact-regression.pt


ESM model loaded.


In [ ]:
def generate_mutant_sequence(mut_str, wt_seq):
    m = re.match(r"([A-Z])(\d+)([A-Z])", mut_str)
    if m:
        orig_letter, pos_str, mut_letter = m.groups()
        pos = int(pos_str)
        mutated_seq = wt_seq[:pos-1] + mut_letter + wt_seq[pos:]
        return mutated_seq, orig_letter, mut_letter, pos
    else:
        return None, None, None, None

def get_batch_embeddings(sequences, batch_converter, model_esm, device, batch_size=8):
    all_embeddings = []
    total = len(sequences)
    num_batches = (total - 1) // batch_size + 1
    for i in range(0, total, batch_size):
        batch_seqs = sequences[i:i+batch_size]
        data = [(str(i+j), seq) for j, seq in enumerate(batch_seqs)]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        batch_tokens = batch_tokens.to(device)
        with torch.no_grad():
            results = model_esm(batch_tokens, repr_layers=[34], return_contacts=False)
        token_representations = results["representations"][34]
        # Average pooling over tokens
        embeddings = token_representations.mean(1).cpu().numpy()
        all_embeddings.append(embeddings)
        print(f"Processed batch {i//batch_size + 1}/{num_batches}")
        # Free up unused GPU memory
        torch.cuda.empty_cache()
    return np.vstack(all_embeddings)

def extract_all_features(df, wt_seq, hydropathy, batch_size=8):
    engineered_features = []
    mutated_seqs = []
    for mut in df['mutant']:
        mutated_seq, orig_letter, mut_letter, pos = generate_mutant_sequence(mut, wt_seq)
        if mutated_seq is None:
            engineered_features.append([np.nan, np.nan, np.nan, np.nan])
            mutated_seqs.append("")
        else:
            aa_diff = ord(mut_letter) - ord(orig_letter)
            hydro_orig = hydropathy.get(orig_letter, np.nan)
            hydro_mut = hydropathy.get(mut_letter, np.nan)
            hydro_diff = hydro_mut - hydro_orig if (hydro_orig is not None and hydro_mut is not None) else np.nan
            rel_pos = pos / len(wt_seq)
            engineered_features.append([pos, aa_diff, hydro_diff, rel_pos])
            mutated_seqs.append(mutated_seq)
    engineered_features = np.array(engineered_features)
    embeddings = get_batch_embeddings(mutated_seqs, batch_converter, model_esm, device, batch_size=batch_size)
    features = np.concatenate((engineered_features, embeddings), axis=1)
    return features

In [ ]:
hydropathy = {
    'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
    'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

with open('sequence.fasta', 'r') as f:
    lines = f.readlines()
    wt_sequence = "".join(line.strip() for line in lines if not line.startswith('>'))
print("Wild-type sequence loaded. Length:", len(wt_sequence))

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
print("\nTraining data preview:")
print(train_df.head())
print("\nTest data preview:")
print(test_df.head())

Wild-type sequence loaded. Length: 656

Training data preview:
  mutant  DMS_score
0    M0Y     0.2730
1    M0W     0.2857
2    M0V     0.2153
3    M0T     0.3122
4    M0S     0.2180

Test data preview:
  mutant
0    V1D
1    V1Y
2    V1C
3    V1A
4    V1E


In [ ]:
print("\nExtracting features for training data using batching...")
train_features = extract_all_features(train_df, wt_sequence, hydropathy, batch_size=8)
print("Training features shape:", train_features.shape)
print("\nExtracting features for test data using batching...")
test_features = extract_all_features(test_df, wt_sequence, hydropathy, batch_size=8)
print("Test features shape:", test_features.shape)
y_train_full = train_df['DMS_score'].values
print("\nSplitting data into training and validation sets...")
X_train, X_val, y_train, y_val = train_test_split(train_features, y_train_full, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape, "| X_val shape:", X_val.shape)


Extracting features for training data using batching...
Processed batch 1/168
Processed batch 2/168
Processed batch 3/168
Processed batch 4/168
Processed batch 5/168
Processed batch 6/168
Processed batch 7/168
Processed batch 8/168
Processed batch 9/168
Processed batch 10/168
Processed batch 11/168
Processed batch 12/168
Processed batch 13/168
Processed batch 14/168
Processed batch 15/168
Processed batch 16/168
Processed batch 17/168
Processed batch 18/168
Processed batch 19/168
Processed batch 20/168
Processed batch 21/168
Processed batch 22/168
Processed batch 23/168
Processed batch 24/168
Processed batch 25/168
Processed batch 26/168
Processed batch 27/168
Processed batch 28/168
Processed batch 29/168
Processed batch 30/168
Processed batch 31/168
Processed batch 32/168
Processed batch 33/168
Processed batch 34/168
Processed batch 35/168
Processed batch 36/168
Processed batch 37/168
Processed batch 38/168
Processed batch 39/168
Processed batch 40/168
Processed batch 41/168
Processed

In [ ]:
# sklearn model
print("\nTraining Gradient Boosting Regressor...")
model = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.05, max_depth=3, random_state=42)
model.fit(X_train, y_train)
print("Model training completed on training split.")



Training Gradient Boosting Regressor...
Model training completed on training split.


In [ ]:
val_preds = model.predict(X_val)
initial_corr, _ = spearmanr(y_val, val_preds)
print("\nInitial Validation Spearman Correlation:", initial_corr)
if initial_corr < 0:
    val_preds = -val_preds
    print("Predictions flipped due to negative correlation.")
    initial_corr, _ = spearmanr(y_val, val_preds)
    print("Validation Spearman Correlation after flipping:", initial_corr)
val_preds_ranked = pd.Series(val_preds).rank(method='dense').values
ranked_corr, _ = spearmanr(y_val, val_preds_ranked)
print("Validation Spearman Correlation (Ranked Predictions):", ranked_corr)


Initial Validation Spearman Correlation: 0.4946213370130747
Validation Spearman Correlation (Ranked Predictions): 0.4946213370130747


In [ ]:
print("\nRetraining model on full training data...")
model.fit(train_features, y_train_full)
test_preds = model.predict(test_features)
train_preds = model.predict(train_features)
train_corr, _ = spearmanr(y_train_full, train_preds)
print("Full Training Spearman Correlation:", train_corr)
if train_corr < 0:
    test_preds = -test_preds
    print("Test predictions flipped based on training correlation.")
test_preds_ranked = pd.Series(test_preds).rank(method='dense').values


Retraining model on full training data...
Full Training Spearman Correlation: 0.9526268768444984


In [ ]:

submission = test_df[['mutant']].copy()
submission['DMS_score_predicted'] = test_preds_ranked
submission.to_csv('predictions.csv', index=False)
print("\nSubmission saved to predictions.csv")



Submission saved to predictions.csv


In [ ]:
import pandas as pd

predictions = pd.read_csv('predictions.csv')

top10 = predictions.sort_values('DMS_score_predicted', ascending=False).head(10)

with open('top10.txt', 'w') as f:
    for mutant in top10['mutant']:
        f.write(f"{mutant}\n")

print("Top 10 mutants saved to top10.txt")


Top 10 mutants saved to top10.txt


In [ ]:
# trying wiht neural network



# ---------------------------------------------------
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

net = Net(input_size=X_train.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

n_epochs = 50
for epoch in range(n_epochs):
    net.train()
    epoch_loss = 0
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        preds = net(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

Epoch 1/50, Loss: 0.1107
Epoch 2/50, Loss: 0.0424
Epoch 3/50, Loss: 0.0461
Epoch 4/50, Loss: 0.0430
Epoch 5/50, Loss: 0.0410
Epoch 6/50, Loss: 0.0403
Epoch 7/50, Loss: 0.0406
Epoch 8/50, Loss: 0.0451
Epoch 9/50, Loss: 0.0455
Epoch 10/50, Loss: 0.0419
Epoch 11/50, Loss: 0.0404
Epoch 12/50, Loss: 0.0398
Epoch 13/50, Loss: 0.0400
Epoch 14/50, Loss: 0.0385
Epoch 15/50, Loss: 0.0410
Epoch 16/50, Loss: 0.0395
Epoch 17/50, Loss: 0.0387
Epoch 18/50, Loss: 0.0412
Epoch 19/50, Loss: 0.0427
Epoch 20/50, Loss: 0.0404
Epoch 21/50, Loss: 0.0396
Epoch 22/50, Loss: 0.0390
Epoch 23/50, Loss: 0.0394
Epoch 24/50, Loss: 0.0391
Epoch 25/50, Loss: 0.0399
Epoch 26/50, Loss: 0.0394
Epoch 27/50, Loss: 0.0420
Epoch 28/50, Loss: 0.0408
Epoch 29/50, Loss: 0.0407
Epoch 30/50, Loss: 0.0394
Epoch 31/50, Loss: 0.0388
Epoch 32/50, Loss: 0.0388
Epoch 33/50, Loss: 0.0384
Epoch 34/50, Loss: 0.0416
Epoch 35/50, Loss: 0.0384
Epoch 36/50, Loss: 0.0405
Epoch 37/50, Loss: 0.0410
Epoch 38/50, Loss: 0.0386
Epoch 39/50, Loss: 0.

In [ ]:
# evlauation

net.eval()
with torch.no_grad():
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    val_preds_nn = net(X_val_tensor).cpu().numpy().ravel()
print("Neural Network Validation Spearman Correlation:", spearmanr(y_val, val_preds_nn)[0])

Neural Network Validation Spearman Correlation: 0.3230852838092417


In [ ]:
# xgboos tmodel

import xgboost as xgb

print("\nTraining XGBoost Regressor...")
# 20.65 % accuracy:
# model = xgb.XGBRegressor(n_estimators=30000, learning_rate=0.005, max_depth=3, random_state=42)
model = xgb.XGBRegressor(n_estimators=5000, learning_rate=0.05, max_depth=3, random_state=42)
model.fit(X_train, y_train)
print("Model training completed on training split.")

# Validation Predictions
val_preds = model.predict(X_val)
initial_corr, _ = spearmanr(y_val, val_preds)
print("\nInitial Validation Spearman Correlation:", initial_corr)
if initial_corr < 0:
    val_preds = -val_preds
    print("Predictions flipped due to negative correlation.")
    initial_corr, _ = spearmanr(y_val, val_preds)
    print("Validation Spearman Correlation after flipping:", initial_corr)
val_preds_ranked = pd.Series(val_preds).rank(method='dense').values
ranked_corr, _ = spearmanr(y_val, val_preds_ranked)
print("Validation Spearman Correlation (Ranked Predictions):", ranked_corr)

print("\nRetraining model on full training data...")
model.fit(train_features, y_train_full)
test_preds = model.predict(test_features)
train_preds = model.predict(train_features)
train_corr, _ = spearmanr(y_train_full, train_preds)
print("Full Training Spearman Correlation:", train_corr)
if train_corr < 0:
    test_preds = -test_preds
    print("Test predictions flipped based on training correlation.")
test_preds_ranked = pd.Series(test_preds).rank(method='dense').values

submission = test_df[['mutant']].copy()
submission['DMS_score_predicted'] = test_preds_ranked
submission.to_csv('predictions.csv', index=False)
print("\nSubmission saved to predictions.csv")



Training XGBoost Regressor...
Model training completed on training split.

Initial Validation Spearman Correlation: 0.6434167885340548
Validation Spearman Correlation (Ranked Predictions): 0.6434167885340548

Retraining model on full training data...
Full Training Spearman Correlation: 0.998927141451332

Submission saved to predictions.csv


In [ ]:
# this is to generate uncertain predictions
import numpy as np

num_models = 5
ensemble_preds = np.zeros((len(test_features), num_models))

for i in range(num_models):
    bootstrap_idx = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_train_boot = X_train[bootstrap_idx]
    y_train_boot = y_train[bootstrap_idx]
    model_boot = xgb.XGBRegressor(n_estimators=5000, learning_rate=0.05,
                                  max_depth=3, random_state=42 + i)
    model_boot.fit(X_train_boot, y_train_boot)

    ensemble_preds[:, i] = model_boot.predict(test_features)

uncertainty = ensemble_preds.std(axis=1)
top_uncertain_indices = np.argsort(-uncertainty)[:100]

# extract the mutants
query_mutants = test_df.loc[top_uncertain_indices, 'mutant'].tolist()

with open("query.txt", "w") as f:
    for mutant in query_mutants:
        f.write(mutant + "\n")

print("\nQuery file 'query.txt' created with the 100 most uncertain mutants.")



Query file 'query.txt' created with the 100 most uncertain mutants.


In [ ]:
import pandas as pd
# loading
predictions = pd.read_csv('predictions.csv')

top10 = predictions.sort_values('DMS_score_predicted', ascending=False).head(10)
with open('top10.txt', 'w') as f:
    for mutant in top10['mutant']:
        f.write(f"{mutant}\n")

print("Top 10 mutants saved to top10.txt")


Top 10 mutants saved to top10.txt
